### 논문
- https://www.techfak.uni-bielefeld.de/~stoye/files/bergeron.pdf

In [230]:
arr = [0, -3, 1, 2, 4, 6, 5, 7, -15, -13, -14, -12, -10, -11, -9, 8, 16]

In [231]:
n = arr[-1]
pi = list(map(abs, arr))
sigma = [v >= 0 for v in arr]

#### 알고리즘

In [241]:
def algorithm_1(pi, sigma):
    """
    Find the components of signed permutation P = (pi, sigma).
    """
    
    M1, M2 = [n], [0] 
    S1, S2 = [0], [0]
    M = [n] + [None] * n
    m = [0] + [None] * n

    component_starts_at = [None] * (n + 1)
    component_ends_at = [None] * (n + 1)
    
    for i in range(1, n + 1):
        """ Compute the M[i] """
        if pi[i - 1] > pi[i]:
            M1.append(pi[i - 1])
        else:
            while M1 and M1[-1] < pi[i]:
                M1.pop()
        M[i] = M1[-1]
        
        """ Find direct components """
        while pi[S1[-1]] > pi[i] or M[S1[-1]] < pi[i]:
            S1.pop()
        s = S1[-1]
        if sigma[i] and M[i] == M[s] and i - s == pi[i] - pi[s]:
            component_starts_at[s] = (s, i)
            component_ends_at[i] = (s, i)
        
        """ Compute the m[i] """
        if pi[i - 1] < pi[i]:
            M2.append(pi[i - 1])
        else:
            while M2 and M2[-1] > pi[i]:
                M2.pop()
        m[i] = M2[-1]
        
        """ Find reversed components """
        while (pi[S2[-1]] < pi[i] or m[S2[-1]] > pi[i]) and S2[-1] > 0:
            S2.pop()
        s = S2[-1]
        if not sigma[i] and m[i] == m[s] and i - s == pi[s] - pi[i]:
            component_starts_at[s] = (s, i)
            component_ends_at[i] = (s, i)
        
        """ Update stacks """
        if sigma[i]:
            S1.append(i)
        else:
            S2.append(i)
    return component_starts_at, component_ends_at

In [242]:
from __future__ import annotations

class Node:
    id_ = 0
    def __init__(
        self,
        is_round: bool | None = None, 
        parent: Node | None = None,
        data: any = None
    ):
        self.id_ = Node.id_
        Node.id_ += 1
        if is_round is None:
            self.is_square = True
            self.is_round = False
        else:
            self.is_square = not is_round
            self.is_round = is_round
        
        self.parent = parent
        self.children = []
        self.data = data
        
        if parent is not None:
            parent.children.append(self)
    
    def __repr__(self):
        return f'Node({self.id_}, [{", ".join(map(repr, self.children))}])'

In [274]:
def algorithm_2(component_starts_at, component_ends_at):
    root = Node()
    q = root
    p = Node(is_round=True, parent=q, data=component_starts_at[0])
    
    for i in range(1, n):
        if C := component_starts_at[i]:
            if not component_ends_at[i]:
                q = Node(parent=p)
            p = Node(is_round=True, parent=q, data=C)
        elif component_ends_at[i]:
            p = q.parent
            q = p.parent
        
    return root

#### 실행

In [275]:
component_starts_at, component_ends_at = algorithm_1(pi, sigma)
tree_root = algorithm_2(component_starts_at, component_ends_at)

In [276]:
def dfs(node, depth=0):
    if node.is_round:
        print('..'*depth + '() ' + str(node.data), end=' ')
        if node.data:
            print(f'({arr[node.data[0]]}, {arr[node.data[1]]})', end='')
        print()
    else:
        print('..'*depth + '[]')
    for child in node.children:
        dfs(child, depth + 1)

dfs(tree_root)

[]
..() (0, 4) (0, 4)
....[]
......() (2, 3) (1, 2)
..() (4, 7) (4, 7)
..() (7, 16) (7, 16)
....[]
......() (8, 11) (-15, -12)
......() (11, 14) (-12, -9)
